# OpenMSI Arrayed Analysis Tool
This Notebook is meant as an example for those who prefer programmatic access to the OpenMSI Arrayed Analysis toolkit over the interactive dialogs in the other notebook

***PLEASE NOTE THIS EXAMPLE IS MEANT FOR EXPERIENCED PYTHON PROGRAMMERS.***

***If you want a no-fuss data analysis tool, use the notebook called <a href="OpenMSI_Arrayed_Analysis_with_dialogs.ipynb">OpenMSI_Arrayed_Analysis_with_dialogs.ipynb</a>.***

There is, of course, more than one way to leverage this tool, so use this example to inspire your analysis pipeline. You might want to mix-and-match interactive commands from the interactive notebook with some non-interactive code featured here.

This way of accessing the toolbox is still not entirely non-interactive, because logging in and spot grid definition (the roughPosition function) are ***always*** interactive

Avoid in-lining matplotlib in this notebook, because the spot placement GUI requires windowed mode.
If you do need to use inline graphics, save the img using pickle, and un-pickle it in a fresh notebook in which you've also %run Arrayed_Analysis_Tools.ipy

In [ ]:
#load the code. Since it's specialized ipython notebook code, use '%run' rather than 'import'
%run Arrayed_Analysis_Tools.ipy

In [ ]:
#OPTIONAL: log into OpenMSI.nersc.gov
openMSIsession = login()

In [ ]:
#If you choose NOT to log in, you can create a new openMSISession
if "openMSIsession" not in locals():
    openMSIsession=OpenMSIsession()

In [ ]:
#Use a file selector to pick a file to download
openMSIsession.fileSelector() #if this is not the last statement of the cell,
                              #you'll need to call IPython.display.display() on this
#fileSelector() stores the picked file in the openMSI object
#so that it downloads the right file in the getArrayedImage function later

In [ ]:
#generates an ArrayedImage from these specificied ions.
myIons = [351.25, 212.05, 137.04]
img=openMSIsession.getArrayedImage(myIons,0.3,massRangeReductionStrategy=PeakArea())
# Look at this function's docstring to find explanations of all the arguments, or use the defaults
# MassReductionStrategies available are PeakArea() PeakHeight() and AreaNearPeak(), PeakArea() is the default
# If you don't like to use the fileselector in the previous cell, you can pass a filename to the getArrayedImage function

In [ ]:
#optional: check what the image looks likes now

plt.imshow(img.baseImage,cmap='jet_r',clim=(0.0,np.amax(img.baseImage)/2)) #get rid of the /2 to see a wider range, or divide
                                                                           #by a bigger number if you want a narrower range
plt.colorbar() 
putWindowOnTop()
plt.show()

In [ ]:
#define spot centers as a trapezoid. Set Hexagonal == True for dense circle packing or False for square packing
columns = 20
rows = 12
img.roughPosition(columns,rows,colormap='jet_r',hexagonal=True)

In [ ]:
#automagically optimize the spot centers to correspond to the actual spots on the image
img.optimizeSpots(verbose=True,minimumScore=300,halfboxsize=2,integrationRadius=2,avoidOverlaps=True,overlapDistance_squared=3)

#if you don't want spots to move off the grid unless they actually find a spot, set a minimumScore.
#(this is useful is there's missing spots from your grid)
#if you want to find out what a reasonable number for a minimumScore is, try running the optimization with verbose=True,
#or use the img.optimizeSpots_with_dialogs() function.
#conversely, if you want to make /sure/ that there is a spot with a score of /at least/ minimumScore,
#you can choose to set raiseExceptions=True

In [ ]:
#check the positions of the spots and manually adjust them if need be
img.fineTunePosition(colormap='jet_r',markerRadius=2,spotLabelsAlwaysOn=True)

In [ ]:
#Optional: Save the ArrayedImage into a pickle file.
import pickle
pickle.dump(img, open("saved_image","wb"))

In [ ]:
#Optional: Load an ArrayedImage from a pickle file. This way you can work off-line
import pickle
img=pickle.load(open("saved_image","rb"))
print img

In [ ]:
#all the rest of the functions can be run non-interactively. In this example it will save the results to a tab-separated file
#as well as to a pandas data-frame for data analysis directly in this notebook

#You'll need to call this function. It returns a list of spots (where each spot is a list of pixels),
#which is also stored inside the object.
spots=img.generateSpotList(integrationRadius=2)

#Write results to a file
#if you don't pass it an explicit spotList it will use the spot set stored in the ArrayedImage
img.writeResultTable("resultTable.csv",minPixelIntensity=1000,alphaRows=True)

df=img.resultsDataFrame(minPixelIntensity=0,alphaRows=True) #generate the dataframe
#alphaRows=True makes the indexes of the data frame strings with an alphabetical row identifier.
#alphaRows=False sets the indexes to 2-tuples (row,column)

IPython.display.display(df)
sums_df=df.loc[:,(slice(None),'sum')] #get the sums from the dataframe
sums_df.columns = sums_df.columns.get_level_values(0) #name the columns to make indexing easier later
percentage_firstion=100.0*sums_df[img.ions[0]]/sums_df.sum(axis=1) #calculate the percentage
percentage_firstion.sort() #rank the data from low to high. Using sort() instead of sort_values(inplace=True) for backward compatibility
plt.bar(range(len(percentage_firstion)),percentage_firstion,edgecolor='b') #define a bar chart
plt.xlabel('Spot rank') #set x axis label
plt.ylabel("% m/z={:.1f} of all loaded ions".format(img.ions[0])) #set y axis label
plt.xlim(0,len(percentage_firstion)-1) #set x axis range
plt.show()



In [ ]:
#Optional: If you want to see a visual respresenation of which pixels are taken into account when
#calculating the result table. If you don't pass it an explicit spotList it will use the stored spotList
maskedimg=img.showMaskedImage(alphaRows=True)
#if you don't want to see the spot row and column co-ordinates, use img.generateMaskedImage() in stead